In [6]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import altair as alt
import lxml

In [7]:
LATEST_GAMEWEEK = 28

Scrape fantrax standings

In [8]:
def buildDF(content):
    table = content.find("ultimate-table")
    names =  table.find("aside").find_all("td")
    rankings = []
    name_list = []
    for td in names:
        rank_and_name = td.text.split("  ")
        rank = rank_and_name[0]
        name = rank_and_name[1]
        rankings.append(rank)
        name_list.append(name)
    data = table.find("table")
    df = pd.read_html(str(data))[0]
    df.columns = ["W","D","L","Points","Win%","WW","FPtsF","FPtsA","Streak"]
    df["team"] = name_list
    df["rank"] = rankings
    df["gw"] = gw
    return df

In [9]:
df_list = []

for i in range(LATEST_GAMEWEEK):

  # open web browser for each gameweek
  gw = i + 1
  url= f"https://www.fantrax.com/fantasy/league/962c1utjlzbm8vb8/standings;view=REGULAR_SEASON;timeframeType=BY_PERIOD;timeStartType=FROM_SEASON_START;period={gw}"
  driver = webdriver.Chrome()
  driver.get(url)

  # wait for the table to load
  try:
    elem = WebDriverWait(driver, 30).until(
      EC.presence_of_element_located((By.TAG_NAME, "ultimate-table")) #This is a dummy element
    )
  finally:
    html = driver.page_source
    soup=BeautifulSoup(html,"html.parser")
    content = soup.find("div", {"class":"league-standings-table"})
    driver.quit()

  df = buildDF(content)

  df_list.append(df)

In [10]:
concat_df = pd.concat(df_list)

In [11]:
concat_df['team'] = concat_df['team'].str.strip()

In [12]:
concat_df['rank'] = concat_df['rank'].str.strip().astype(int)

In [13]:
data = concat_df

chart = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, 38]), title="Gameweek"),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[10,1]), title="Rank"),
    color=alt.Color("team", legend=None),
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
).properties(
    title="Fake Internet Soccer XIV standings by gameweek",
    width=700,
    height=350,
)

labels = alt.Chart(data).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, 38])),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[10,1])),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
)

chart + labels

alt.LayerChart(...)

In [14]:
chart = alt.Chart(concat_df).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, 38]), title="Gameweek"),
    y=alt.Y("Points", title="Points"),
    color=alt.Color("team", legend=None),
    tooltip=["team"]
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="descending")],
    groupby=["gw"]
).properties(
    title="Fake Internet Soccer XIV standings by gameweek",
    width=700,
    height=350,
)

labels = alt.Chart(concat_df).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, 38])),
    y=alt.Y('Points', aggregate={'argmax': 'gw'}),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="descending")],
    groupby=["gw"]
)

chart + labels

alt.LayerChart(...)

In [21]:
concat_df.to_csv("data/output/standings/standings-season-14.csv", index=False)

To JSON

In [16]:
clean_df = concat_df.reset_index()[["team","rank","gw","Points"]]

In [17]:
clean_df.columns = clean_df.columns.str.lower()

In [18]:
clean_df.to_json("../_data/standings-xiv.json", orient='records')